In [ ]:
#! pip install streamz

Imports

In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.models import LabelSet
import datetime as dt
import panel as pn
import concurrent.futures
from bokeh.models import ColumnDataSource
pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df
from datetime import datetime, timezone
import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d
from datetime import date, time
import random
import numpy as np
import pandas as pd



import hvplot.streamz
import holoviews as hv
from streamz import Stream
from streamz.dataframe import DataFrame

import panel as pn
from time import sleep
from bokeh.models.formatters import DatetimeTickFormatter
pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

### Configure cognite client

In [ ]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"
# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"


from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter



### Define Dataset

In [ ]:
#  
wellcom_dataset = "src:015:wellcom"
witsml_dataset = "src:014:witsmldata"
active_wellbore = "active:wellbore"

#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
#Get sensor data 
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)

# Define Run plot

In [ ]:

def run_plot(c):
    try:
        c.start()
    except Exception as e:
        print(f"error {e}")
        c.stop()
    finally:
        
        print(f"state of c is {c.running}")
        c.stop()

# real time fake data

In [ ]:
#05.08

# Set up streaming DataFrame
stream = Stream() # init stream
df_ = pd.DataFrame({'a': 69}, index= [dt.datetime(2024, 8, 4, 00, 00, 00, tzinfo=timezone.utc) ])
        
# samples = pd.DataFrame({'x':[0],'y':[0]})

df_stream = DataFrame(stream, example=df_)

# declare plots
plot_test = df_stream.hvplot(kind='scatter')

# set up callback with streaming data
def emit_fake():
    global df_stream
    try:

        
        df_new = pd.DataFrame({'a': np.random.randint(1, 10, 1)}, index=[dt.datetime.now()])
        
        print("Hehe new line ")

        # if (len(stream.to_dataframe()) > 5):
        #     print("Stopping now")

        df_stream.emit(df_new)
        print(df_new)
    except Exception as e:
        print(f"error {e}")
    finally:
        print("Step finished.")




# emit_fake()
# #Plot
pn.serve(pn.Row(plot_test))
# # plot_test
# # callback
cb = pn.state.add_periodic_callback(emit_fake, 5000, timeout=30000)



# Real time real data

In [65]:
#05.08

# Set up streaming DataFrame
stream = Stream() # init stream


df_ = client.time_series.data.retrieve(id=368728207134017,start=dt.datetime(2024, 8, 5, 00, 00, 00, tzinfo=timezone.utc),  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
df_ = df_.to_pandas().sort_index(ascending=False)  
df_['Value'] = df_[df_.columns[0]]
df_['Date'] = pd.to_datetime(df_.index)
        
# samples = pd.DataFrame({'x':[0],'y':[0]})

df_stream = DataFrame(stream, example=df_)

# declare plots
plot_test = df_stream.hvplot(kind='scatter',x='Value', y ='Date')

# set up callback with streaming data
def emit_fake():
    global df_stream
    try:

        
        # df_new = pd.DataFrame({'a': np.random.randint(1, 10, 1)}, index=[dt.datetime.now()])
        
        df_new = client.time_series.data.retrieve(id=368728207134017,start=dt.datetime(2024, 8, 5, 00, 00, 00, tzinfo=timezone.utc),  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
        df_new = df_new.to_pandas().sort_index(ascending=False)  
        df_new['Value'] = df_new[df_new.columns[0]]
        df_new['Date'] = pd.to_datetime(df_new.index)

        print("Hehe new line ")

        # if (len(stream.to_dataframe()) > 5):
        #     print("Stopping now")

        df_stream.emit(df_new)
        print(df_new)
    except Exception as e:
        print(f"error {e}")
    finally:
        print("Step finished.")




# emit_fake()
# #Plot
pn.serve(pn.Row(plot_test))
# # plot_test
# # callback
cb = pn.state.add_periodic_callback(emit_fake, 5000, timeout=600000)



Launching server at http://localhost:62260


Hehe new line 
                     dw:witsml:log.2b328fe0-5e80-4d9b-ad6f-56a4cb591371.d08c0bb9-c45f-46ec-b497-c682d6f7aac6.d08c0bb9-c45f-46ec-b497-c682d6f7aac6.bitdep||v1  \
2024-08-05 13:21:24                                        4133.375900                                                                                         
2024-08-05 13:21:23                                        4133.375900                                                                                         
2024-08-05 13:21:22                                        4133.375900                                                                                         
2024-08-05 13:21:21                                        4133.375900                                                                                         
2024-08-05 13:21:20                                        4133.375900                                                                                         
...                      

In [70]:
#05.08 REAL

# Set up streaming DataFrame
stream = Stream() # init stream

_glb_datetime = dt.datetime(2024, 8, 5, 00, 00, 00, tzinfo=timezone.utc)
df_ = client.time_series.data.retrieve(id=368728207134017,start=_glb_datetime,  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
df_ = df_.to_pandas().sort_index(ascending=False)  
df_['Value'] = df_[df_.columns[0]]
df_['Date'] = pd.to_datetime(df_.index)
        
# samples = pd.DataFrame({'x':[0],'y':[0]})

df_stream = DataFrame(stream, example=df_)

# declare plots
plot_test = df_stream.hvplot(kind='scatter',x='Value', y ='Date')

# set up callback with streaming data
def emit_fake():
    global df_stream, _glb_datetime
    try:

        
        # df_new = pd.DataFrame({'a': np.random.randint(1, 10, 1)}, index=[dt.datetime.now()])
        
        df_new = client.time_series.data.retrieve(id=368728207134017,start=_glb_datetime,  end=MAX_TIMESTAMP_MS)#ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
        df_new = df_new.to_pandas().sort_index(ascending=False)  
        df_new['Value'] = df_new[df_new.columns[0]]
        df_new['Date'] = pd.to_datetime(df_new.index)

        print("Hehe new line ")

        # make new datetime
        last_time = df_new['Date'].max() # Update the last timestam
        dt_obj = dt.datetime.strptime(str(last_time), '%Y-%m-%d %H:%M:%S')
        year = dt_obj.year
        month = dt_obj.month
        day = dt_obj.day
        hour = dt_obj.hour
        minute = dt_obj.minute
        second = dt_obj.second
        new_time = dt.datetime(year, month, day, hour, minute, second, tzinfo=timezone.utc)
        # try:
        #     (new_time > timesss)
        #     print(f" new_time {new_time}, timesss {timesss}")
        # except Exception as e:
        #     print(f"error : {e}")

        if (new_time > _glb_datetime):

        # if (len(stream.to_dataframe()) > 5):
        #     print("Stopping now")
            print(df_new)
            df_stream.emit(df_new)
            print(f"new_time is {new_time} while _glb_datetime is {_glb_datetime}")
            _glb_datetime = new_time

    except Exception as e:
        print(f"error {e}")
    finally:
        print("Step finished.")




# emit_fake()
# #Plot
pn.serve(pn.Row(plot_test))
# # plot_test
# # callback
cb = pn.state.add_periodic_callback(emit_fake, 5000, timeout=600000)



Launching server at http://localhost:62588


Hehe new line 
Step finished.
Hehe new line 
Step finished.
Hehe new line 
                     dw:witsml:log.2b328fe0-5e80-4d9b-ad6f-56a4cb591371.d08c0bb9-c45f-46ec-b497-c682d6f7aac6.d08c0bb9-c45f-46ec-b497-c682d6f7aac6.bitdep||v1  \
2024-08-05 13:30:57                                        4133.375900                                                                                         
2024-08-05 13:30:56                                        4133.375900                                                                                         
2024-08-05 13:30:55                                        4133.375900                                                                                         
2024-08-05 13:30:54                                        4133.375900                                                                                         
2024-08-05 13:30:53                                        4133.375900                                                       